# Collection setup and data load

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
openai_key = os.getenv("OPENAI_API_KEY")
openai_url = os.getenv("OPENAI_URL")

print(weaviate_url, openai_key, openai_url)

## Connect to Weaviate

You need to pass in your OpenAI key, which will be used to vectorise your data.

In [ ]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_local(
    host=weaviate_url,

    headers={
        "X-OpenAI-Api-Key": openai_key,
    },

    # connect with timeout config
    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
    # )
)

client.is_ready()

## Create a collection with a vectorizer

* [Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/manage-data/collections)
* [OpenAI integrated embedding models](https://weaviate.io/developers/weaviate/model-providers/openai/embeddings)

Examples of other embedding models:
* [Cohere](https://weaviate.io/developers/weaviate/model-providers/cohere/embeddings)
* [HuggingFace 🤗](https://weaviate.io/developers/weaviate/model-providers/huggingface/embeddings)
* [Ollama (self-hosted)](https://weaviate.io/developers/weaviate/model-providers/ollama/embeddings)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with OpenAI vectorizer
client.collections.create(
    name="Jeopardy",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small",
        base_url=openai_url,
    )
)

## Import data
### Sample Data

In [ ]:
import json

with open("./jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

### Insert Many

> `insert_many` is only used for inserting small batches of data - must complete within the timeout.

[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_10)

### Data preview

In [ ]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    print(item.uuid, item.properties)

In [ ]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

### Super quick query example

In [ ]:
response = jeopardy.query.near_text(
    query="African animals",
    # query="weather",
    limit=2
)

for item in response.objects:
    print(item.properties)

## A bit bigger example - 20k objects

### Load data

In [ ]:
import json

with open("./wiki-20k.json") as file:
    data_20k = json.load(file)

print(json.dumps(data_20k[0:2], indent=2))

### Create a collection with Named Vectors and SourceProperties

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with OpenAI vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vectorizer_config=[
            Configure.NamedVectors.text2vec_openai(
                name="main_vector",

                model="text-embedding-3-small",
                base_url=openai_url,

                source_properties=['title', 'text'] # which properties should be used to generate a vector
            )
        ],

        # Example: how to define property schema (Optional)
        # properties=[  
        #     Property(name="title", data_type=DataType.TEXT),
        #     Property(name="text", data_type=DataType.TEXT),
        #     Property(name="url", data_type=DataType.TEXT),
        #     Property(name="wiki_id", data_type=DataType.TEXT),
        # ],
    )

create_wiki_collection()

### Import data - 20k objects with Batch

Batch speeds up the import process by grouping objects to be added in bigger batch groups.

Batch creates an internal buffer to collect objects to be added.<br>
Each time the buffer count reaches `batch_size`, batch sends the new objects to Weaviate.

Types of batch:
* `dynamic` - let batch calculate the optimal batch_size based on detected latency
* `fixed_size` - provide a fixed batch_size
* `rate_limit` - limit the number of requests (per minute), useful for working with models with a rate limit

### Take 1 – import sample 100

In [ ]:
from tqdm import tqdm

sample_100 = data_20k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.dynamic() as batch:
    for item in tqdm(sample_100):
        batch.add_object(item)

print(f"Wiki count: {len(wiki)}")

In [ ]:
# check for errors
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

### Take 2 – import sample 100 – with UUID

To avoid inserting duplicates, you can generate a UUID based on the whole object or a unique property.

In [ ]:
from weaviate.util import generate_uuid5

print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print("====================================")

print(generate_uuid5("This UUID is different"))
print(generate_uuid5("This UUID is different"))
print("====================================")

obj1 = { "title": "this is an object", "count": 1 }
obj2 = { "title": "this is an object", "count": 2 }
print(generate_uuid5(obj1))
print(generate_uuid5(obj2))


In [ ]:
# recreate the collection to start again
create_wiki_collection()

> Rerun the import script multiple times.

> Starting from the second run, the script should finish a lot faster, and the wiki count shouldn't increase.

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

sample_100 = data_20k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=20, concurrent_requests=2) as batch:
    for item in tqdm(sample_100):
        id = generate_uuid5(item["wiki_id"])

        batch.add_object(
            item,
            uuid=id
        )

print(f"Wiki count: {len(wiki)}")

### Take 2 - import the rest of the data - but break if multiple errors

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=2500, concurrent_requests=4) as batch:
    for item in tqdm(data_20k):
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(item, uuid=id)

        # Check number of errors while running
        if(batch.number_errors > 10):
            print("Errors during batch import")
            break

### Check for errors

In [ ]:
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

## Bonus - iterate through all collection data

The client has a built-in function that allows you to iterate through all collection data.

In [ ]:
wiki = client.collections.get("Wiki")

counter = 100

for item in wiki.iterator():
    print(item.properties)

    if (counter == 0): break
    
    counter -= 1

You can also get `vector embeddings`, by using `include_vector`.

In [ ]:
counter = 10

for item in wiki.iterator(include_vector=True):
    print(item.properties)
    print(item.vector)

    if (counter == 0): break
    
    counter -= 1

## Close the client

In [ ]:
client.close()